## Import the medical note

Run this code to import the dataset

In [3]:
import pandas as pd

# As an example for this test, we use the discharge medical note
# discharge_gz = 'C:/3163dataset/discharge.csv.gz'
# discharge = pd.read_csv(discharge_gz, compression='gzip')

## NOTE: below code for Alex's bughunting
discharge = pd.read_csv("discharge.csv")

Create a variable to store a single patient's medical notes

In [5]:
patient_admission_notes = discharge.loc[discharge['subject_id'] == 10000032]['text']
patient_admission_notes

0     \nName:  ___                     Unit No:   _...
1     \nName:  ___                     Unit No:   _...
2     \nName:  ___                     Unit No:   _...
3     \nName:  ___                     Unit No:   _...
Name: text, dtype: object

## NLP stage 

### BART Model
- This uses Hugging Face Transformers' BART model (standard summarizer model)
- Install the packages: transformes AND pytorch

Credits to: https://blog.gopenai.com/simplifying-healthcare-text-summarization-of-medical-notes-with-python-391c3a1e738d

In [6]:
# This variable stores only one medical note
note = patient_admission_notes[0]

# INSTALL transformers AND pytorch beforehand
from transformers import BartForConditionalGeneration, BartTokenizer
import torch

# BART model
model_name = "facebook/bart-large-cnn"
model = BartForConditionalGeneration.from_pretrained(model_name)
tokenizer = BartTokenizer.from_pretrained(model_name)

# Tokenize the medical note using the BART model and summarise the note
inputs = tokenizer.encode("summarize: " + note, return_tensors="pt", max_length=1024, truncation=True)
summary_ids = model.generate(inputs, max_length=500, min_length=200, length_penalty=2.0, num_beams=4, early_stopping=True)
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print(summary)

/Users/alex/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


Pt has HCV cirrhosis c/b ascites, hiv on ART, h/o IVDU, COPD, PTSD, PTSD. She reported self-discontinuing lasix and spirnolactone because she feels like "they don't do anything" and that she "doesn't want to put more chemicals in her" In the past week, she notes that she                 has been having worsening abd distension and discomfort. She denies easy bruising, melena, BRBPR,                 hemetesis, hemoptysis, or hematuria. She also had a skin lesion, which was biopsied and showed  skin cancer per patient report. She is not aware of any liver disease or liver disease in her family. Her last alcohol consumption was one drink two months ago. She had food poisoning a week ago from eating stale                 cake (n/v 20 min after food ingestion) She denies other recent illness or sick contacts.


### The result using BART model

Pt has HCV cirrhosis c/b ascites, hiv on ART, h/o IVDU, COPD, PTSD, PTSD. She reported self-discontinuing lasix and spirnolactone because she feels like "they don't do anything" and that she "doesn't want to put more chemicals in her" In the past week, she notes that she                 has been having worsening abd distension and discomfort. She denies easy bruising, melena, BRBPR,                 hemetesis, hemoptysis, or hematuria. She also had a skin lesion, which was biopsied and showed  skin cancer per patient report. She is not aware of any liver disease or liver disease in her family. Her last alcohol consumption was one drink two months ago. She had food poisoning a week ago from eating stale                 cake (n/v 20 min after food ingestion) She denies other recent illness or sick contacts.

- (+) It was able to summarise the very long medical note to a single paragraph in 41.3 seconds
- (-) It is not a scientifically-trained model

### BioGPT

In [8]:
# Alex - added in pip installation for sacremoses to try to fix error for cell below
%pip install -U sacremoses

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 2.3 MB/s eta 0:00:00a 0:00:01m
Note: you may need to restart the kernel to use updated packages.


In [12]:
# This variable stores only one medical note
note = patient_admission_notes[0]

from transformers import AutoTokenizer, BioGptForCausalLM

# BioGPT
tokenizer = AutoTokenizer.from_pretrained("microsoft/biogpt")
model = BioGptForCausalLM.from_pretrained("microsoft/biogpt")

# Tokenization and summarisation
inputs = tokenizer.encode("summarize: " + note, return_tensors="pt", max_length=1024, truncation=True)
summary_ids = model.generate(inputs, max_length=1025, min_length=200, length_penalty=2.0, num_beams=4, early_stopping=True)
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print(summary)

summarize: Name: _ _ _ Unit No: _ _ _ Admission Date: _ _ _ Discharge Date: _ _ _ Date of Birth: _ _ _ Sex: F Service: MEDICINE Allergies: No Known Allergies / Adverse Drug Reactions Attending: _ _ _ Chief Complaint: Worsening ABD distension and pain Major Surgical or Invasive Procedure: Paracentesis History of Present Illness: _ _ _ HCV cirrhosis c / b ascites, hiv on ART, h / o IVDU, COPD, bioplar, PTSD, presented from OSH ED with worsening abd distension over past week. Pt reports self-discontinuing lasix and spirnolactone _ _ _ weeks ago, because she feels like "they don't do anything" and that she "doesn't want to put more chemicals in her." She does not follow Na-restricted diets. In the past week, she notes that she has been having worsening abd distension and discomfort. She denies _ _ _ edema, or SOB, or orthopnea. She denies f / c / n / v, d / c, dysuria. She had food poisoning a week ago from eating stale cake (n / v 20 min after food ingestion), which resolved the same day.

## Generated Sumamry Output with BioGPTForCasualLm

---

summarize: Name: _ _ _ Unit No: _ _ _ Admission Date: _ _ _ Discharge Date: _ _ _ Date of Birth: _ _ _ Sex: F Service: MEDICINE Allergies: No Known Allergies / Adverse Drug Reactions Attending: _ _ _ Chief Complaint: Worsening ABD distension and pain Major Surgical or Invasive Procedure: Paracentesis History of Present Illness: _ _ _ HCV cirrhosis c / b ascites, hiv on ART, h / o IVDU, COPD, bioplar, PTSD, presented from OSH ED with worsening abd distension over past week. Pt reports self-discontinuing lasix and spirnolactone _ _ _ weeks ago, because she feels like "they don't do anything" and that she "doesn't want to put more chemicals in her." She does not follow Na-restricted diets. In the past week, she notes that she has been having worsening abd distension and discomfort. She denies _ _ _ edema, or SOB, or orthopnea. She denies f / c / n / v, d / c, dysuria. She had food poisoning a week ago from eating stale cake (n / v 20 min after food ingestion), which resolved the same day. She denies other recent illness or sick contacts. She notes that she has been noticing gum bleeding while brushing her teeth in recent weeks. she denies easy bruising, melena, BRBPR, hemetesis, hemoptysis, or hematuria. Because of her abd pain, she went to OSH ED and was transferred to _ _ _ for further care. Per ED report, pt has brief period of confusion - she did not recall the ultrasound or bloodwork at osh. She denies recent drug use or alcohol use. She denies feeling confused, but reports that she is forgetful at times. In the ED, initial vitals were 98.4 70 106 / 63 16 97% RA Labs notable for ALT / AST / AP _ _ _ _ _ _: _ _ _, Tbili1.6, WBC 5K, platelet 77, INR 1.6 Past Medical History: 1. HCV Cirrhosis 2. No history of abnormal Pap smears. 3. She had calcification in her breast, which was removed previously and per patient not, it was benign. 4. For HIV disease, she is being followed by Dr. _ _ _ Dr. _ _ _. 5. COPD 6. Past history of smoking. 7. She also had a skin lesion, which was biopsied and showed skin cancer per patient report and is scheduled for a complete removal of the skin lesion in _ _ _ of this year. 8. She also had another lesion in her forehead with purple discoloration. It was biopsied to exclude the possibility of _ _ _'s sarcoma, the results is pending. 9. A 15 mm hypoechoic lesion on her ultrasound on _ _ _ and is being monitored by an MRI. 10. History of dysplasia of anus in _ _ _. 11. Bipolar affective disorder, currently manic, mild, and PTSD. 12. History of cocaine and heroin use. Social History: _ _ _ Family History: She a total of five siblings, but she is not talking to most of them. She only has one brother that she is in touch with and lives in _ _ _. She is not aware of any known GI or liver disease in her family. Her last alcohol consumption was one drink two months ago. No regular alcohol consumption. Last drug use _ _ _ years ago. She quit smoking a couple of years ago. Physical Exam: VS: 98.1 107 / 61 78 18 97RA General: in NAD HEENT: CTAB, anicteric sclera, OP clear Neck: supple, no LAD CV: RRR, S1S2, no m / r / g Lungs: CTAb, prolonged expiratory phase, no w / r / r Abdomen: distended, mild diffuse tenderness, + flank dullness, cannot percuss liver / spleen edge _ _ _ distension GU: no foley Ext: wwp, no c / e / e, + clubbing Neuro: AAO3, converse normally, able to recall 3 times after 5 minutes, CN II-XII intact Discharge: PHYSICAL EXAMINATION: VS: 98 105 / 70 95 General: in NAD HEENT: anicteric sclera, OP clear Neck: supple, no LAD CV: RRR, S1S2, no m / r / g Lungs: CTAb, prolonged expiratory phase, no w / r / r Abdomen: distended but improved, TTP in RUQ, GU: no foley Ext: wwp, no c / e / e, + clubbing Neuro: AAO3, CN II-XII intact Pertinent Results: _ _ _ 10: 25PM GLUCOSE-109 * UREA N /


Multiple patient notes

In [ ]:
# TO-DO